In [3]:
import sys, os, git
import numpy as np

def visualize_block(RST):
    prev_block = np.min(RST.main_block)
    for block, type, sub, sub_type, line in zip(RST.main_block, RST.main_block_type,
                                      RST.sub_block, RST.sub_block_type,
                                      RST.file_content):
        if block != prev_block:
            print()
        print(sub, type, sub_type, line)
        assert np.round(block) == np.round(sub), """Strange sub-block numbering"""
        prev_block = block

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")

from testslist import tests

sys.path.append(git_path + "/functions/")

from ReadRST import ReadRST
from WriteTEX import WriteTex
from FixDocument import FixDocument

if os.path.exists(git_path+'/test/tex/converted_files') is False:
    os.mkdir(git_path+'/test/tex/converted_files')

In [4]:
for level in tests.keys():
    if os.path.exists(git_path+'/test/tex/converted_files/'+level) is False:
        os.mkdir(git_path+'/test/tex/converted_files/'+level)
    for tutorial in tests[level]:
        rst_path = git_path+'/test/rst/'+level+'/'
        rst_file_name = rst_path+tutorial+'.rst'
        tex_file_name = git_path+'/test/tex/converted_files/'+level+'/'+tutorial+'.tex'   
        # ######## #
        # READ RST #
        # ######## #
        RST = ReadRST(rst_file_name, rst_path)
        # STEP 1: read the rst file        
        RST.read_rst()
        # STEP 2: add extra content
        RST.include_extra()
        # STEP 3: detect the blocks 
        RST.detect_blocks()
        assert len(RST.main_block) == len(RST.file_content)
        # STEP 4: detect the sub blocks
        RST.detect_sub_blocks()
        # STEP 4: detect the titles
        RST.detect_title()
        # STEP 5: detect the label positions
        RST.detect_label_position()

        # visualize_block(RST)
        
        assert len(RST.label_positions) == 1, """Careful, more than one label"""
        # visualize_block(RST)
        # ######### #
        # WRITE TEX #
        # ######### #
        TEX = WriteTex(tex_file_name, RST, git_path, rst_path)
        TEX.open_f()
        # STEP 1: titles    
        TEX.write_main_title()
        # STEP 2: label
        TEX.write_main_label()
        # STEP 3: blocks
        TEX.loop_on_block()
        TEX.close_f()

        # ####### #
        # FIX DOC #
        # ####### #
        FIX = FixDocument(tex_file_name)
        FIX.remove_space()
        # add no indent
        keywords = [r'\end{lcverbatim}', r'\Large', r'\section{',
                    r'\section*{', r'\subsection*{', r'\label{',
                    r'\subsection{', r'\end{wrapfigure}', r'\end{tcolorbox}',
                    r'\end{figure}']
        FIX.add_non_indent(keywords)
        FIX.convert_itemize()
        FIX.fix_label()
        FIX.fix_external_link()
        FIX.detect_legend()
        FIX.write_legend()
        FIX.remove_space()
        keywords = [r'\noindent', r'\section', r'\subsection', r'\begin',
                    r'\begin{graytitle}', r'\end', r'\label', r'$$']
        FIX.add_vspace(keywords, '0.25cm')